In [ ]:
# 衛星影像判釋水域覆蓋率公開程式碼
# - - - - - - - - - - -
"""
用於台灣本島範圍之監測範圍水域覆蓋率估算。
"""
# 內容
# - - - - - - - - - - -
"""
Part 1 - 下載衛星影像
> 衛星影像參數：
    "type": "sentinel-1-grd",
    "Operational mode": "IW",
    "Resolution": "HIGH",
    "Polarization": "DV",
    "Orbit Direction": "DESCENDING",
    "Polarisation channels"："VV","VH"
> 影像預處理：
  "backCoeff": "GAMMA0_TERRAIN",
  "orthorectify": "true",
  "demInstance": "COPERNICUS",
  "radiometricTerrainOversampling": 2

Part 2 - 接合影像
如果監測範圍寬、高大於25公里，必須將影像切割後，方能下載。
若樣區可能遭裁切於多張影像中，將造成後續水域計算有誤，建議將影像接合後方能進行運算。

Part 3 - 水域覆蓋率判釋
> 水域判釋極化：VV
> 水域判釋數位數值閾值：0.029
"""
# 提醒
# - - - - - - - - - - -
"""
*** 請將程式碼複製一份至新的colab頁面執行
*** 若該段落程式碼執行時間過長，請嘗試將該段落切分成多段依序執行 ***
*** 若無需將每個樣區之sar圖層分別存取，盡量將監測範圍涵蓋於單一polygon中下載衛星影像，避免各樣區影像重疊，影響覆蓋率計算 ***
"""


In [ ]:
# @title Part 1 - Sentinel-1 衛星影像下載

# === 連接 Google Drive ===
"""
我們使用 Google Drive 作為衛星影像及地理空間資訊存取路徑，
故須先連結使用者Google Drive帳號以執行後續作業。
"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# === 載入所需套件 ===
import os, json, requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
from pyproj import Transformer
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session


# === Copernicus OAuth 驗證 ===
"""
請先於 Copernicus Data Space Ecosystem 官網註冊 OAuth client，
並將 client_id、client_secret 填入下方。
詳細操作方法：
https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html#python
"""

def loadtoken():
  # Your client credentials
  client_id = 'client_id' # change to your client_id
  client_secret = 'secret'   # change to your secret

  try:
      # Create a session
      client = BackendApplicationClient(client_id=client_id)
      oauth = OAuth2Session(client=client)

      # Get token for the session
      token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                                client_secret=client_secret, include_client_id=True)

      # All requests using this session will have an access token automatically added
      resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
      print("✅ Copernicus OAuth 驗證成功")
      return oauth
  except Exception as e:
      print(f"❌ Copernicus 驗證失敗：{e}")
      raise


# === 切換工作目錄 ===
"""
將欲存放衛星影像的 Google Drive 資料夾路徑設為新的目錄
"""
geotiff_path = '衛星影像存放路徑'
os.makedirs(geotiff_path, exist_ok=True)
os.chdir(geotiff_path)


# === 定義下載影像參數 ===
"""
修改可參考：
https://docs.sentinel-hub.com/api/latest/reference/
https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S1GRD.html#s1grd-orthorectified-linear-gamma0-vv-between-0-and-05-in-approximate-real-world-10-m-resolution-iw-png
"""
def requestdata(polygon, target, width, height, dt_obj, outputformat="image"):
    dt_obj_end = dt_obj + timedelta(days=1)
    url = "https://sh.dataspace.copernicus.eu/api/v1/process"

    if outputformat == "image":
        evalscript = """
        //VERSION=3
        function setup() {
          return {
            input: ["VV","VH"],
            output: { id: "default", bands: 2, sampleType: SampleType.FLOAT32 }
          }
        }
        function evaluatePixel(samples) {
          return [samples.VV, samples.VH]
        }
        """
        responses = [{"identifier": "default", "format": {"type": "image/tiff"}}]
        processing = {
          "backCoeff": "GAMMA0_TERRAIN",
          "orthorectify": "true",
          "demInstance": "COPERNICUS",
          "radiometricTerrainOversampling": 2
        }
    else:
        evalscript = """
        //VERSION=3
        function setup() {
          return {
            input: ["VV","VH"],
            output: { id: "default", bands: 2, sampleType: SampleType.FLOAT32 },
            mosaicking: "ORBIT"
          }
        }
        function evaluatePixel(samples) {
          return [samples.VV, samples.VH]
        }
        function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {
          outputMetadata.userData = { "scenes": scenes.orbits }
        }
        """
        responses = [{"identifier": "userdata", "format": {"type": "application/json"}}]
        processing = {}

    payload = {
        "input": {
            "bounds": {"geometry": {"type": "Polygon", "coordinates": polygon}},
            "data": [{
                "type": "sentinel-1-grd",
                "dataFilter": {
                    "timeRange": {
                        "from": dt_obj.strftime("%Y-%m-%dT%H:%M:%SZ"),
                        "to": dt_obj_end.strftime("%Y-%m-%dT%H:%M:%SZ"),
                    },
                    "acquisitionMode": "IW",
                    "resolution": "HIGH",
                    "polarization": "DV",
                    "orbitDirection": "DESCENDING"
                },
                "processing": processing
            }]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": responses
        },
        "evalscript": evalscript
    }

    savepath = f"{target}.tif" if outputformat == "image" else f"{target}.json"

    try:
        headers = {"Accept": "image/tiff" if outputformat == "image" else "application/json"}
        response = oauth.post(url, json=payload, headers=headers)
        response.raise_for_status()

        with open(savepath, 'wb') as f:
            f.write(response.content)

        print(f"✅ {savepath} 下載完成")

        if outputformat == "json":
            metadata = response.json()
            print("📅 影像日期：", metadata['scenes'][0]['tiles'][0]['date'])

    except Exception as e:
        print(f"⚠️ 下載失敗：{savepath}，原因：{e}")


# === 匯入監測範圍 ===
"""
上傳 GPKG 或 SHP 監測範圍檔案，
檔案須有一識別各polygon的名稱欄(如"ID"、"name")，作為下載範圍對應欄位。
Sentinel Hub提供使用者下載空間解析度為每像素 10×10m、最大尺寸 2500×2500 像素的單一影像檔案。
若無需將每個樣區之SAR圖層分別存取，且完整監測範圍小於 25,000×25,000m，
可將所有監測範圍涵蓋於單一polygon中，下載一張影像即可。
"""
polygons_path = '監測範圍名稱及路徑'
polygons = gpd.read_file(polygons_path)
if polygons.crs != "EPSG:4326":  # 如果坐標系不是 EPSG:4326，則進行轉換
  polygons = polygons.to_crs("EPSG:4326")
polygons["ID"] = polygons["name"].astype(str) # 填入 polygon 名稱欄

# 計算每個 polygon 的長寬
polygons_m = polygons.to_crs(epsg=3826)
bounds = polygons_m.bounds
polygons["width_10m"] = (bounds["maxx"] - bounds["minx"]) / 10
polygons["height_10m"] = (bounds["maxy"] - bounds["miny"]) / 10

# 建立每個 polygon 的 bbox 座標 (EPSG:4326)
polygons["bbox_coords"] = polygons.geometry.apply(
    lambda g: [[
        [g.bounds[0], g.bounds[1]],
        [g.bounds[0], g.bounds[3]],
        [g.bounds[2], g.bounds[3]],
        [g.bounds[2], g.bounds[1]],
        [g.bounds[0], g.bounds[1]]
    ]]
)

print(f"🗺️ 已讀取 {len(polygons)} 個監測範圍")

# === 衛星影像下載週期 ===
"""
1. 起始日(date_from)請查明衛星拍攝日，填入「UTC+0」之日期。
2. 若僅需單天影像，結束日(date_until)與起始日相同即可。
3. Sentinel-1至今共發射三顆任務衛星，單顆衛星影像週期為12日，預設週期為6或12日下載一組影像。
(S1A 2014至今；S1B 2016至2022；S1C 2025/03/25起提供台灣區影像，任務期程詳情請參考：
https://sentiwiki.copernicus.eu/web/s1-mission）
"""
date_from = datetime(2025, 10, 14)   # 起始日 (UTC+0)
date_until = datetime(2025, 10, 14)  # 結束日 (UTC+0)
date_range = (date_until - date_from).days + 1 # 區間日數
print(f"⏳ 將下載 {date_range} 天範圍內的影像")

oauth = loadtoken()

for d in range(0, date_range, 6):  # 設定衛星週期天數
    new_dt = date_from + timedelta(days=d)  # (UTC+0)
    new_dt_tw = new_dt + timedelta(days=1)  # 對應的台灣時間　(UTC+8)
    for index, region in polygons.iterrows():
        polygon = region["bbox_coords"] # 下載涵蓋polygon之完整矩形像素數值
        # polygon = [list(region.geometry.exterior.coords)] # 僅下載polygon範圍像素數值(與前項擇一使用)
        target = f"S1_DV_SouthwestTaiwan_{new_dt_tw.strftime('%Y-%m-%d')}_{region['ID']}" # 填入 polygon 名稱欄
        width = region["width_10m"]
        height = region["height_10m"]
        requestdata(polygon, target, width, height, new_dt, "image")


print("\n🎉 全部影像下載完成！")
print(f"📁 輸出路徑：{geotiff_path}")

In [ ]:
# @title Part 2 - 衛星影像接合
"""
如果監測範圍寬、高大於25公里，必須將影像切割後，方能下載。
若樣區可能遭裁切於多張影像中，將造成後續水域計算有誤，建議將影像接合後方能進行運算。
"""
# === 連接 Google Drive ===
"""
我們使用 Google Drive 作為衛星影像及地理空間資訊存取路徑，
故須先連結使用者Google Drive帳號以執行後續作業。
"""
from google.colab import drive
drive.mount('/content/drive')


# === 載入所需套件 ===
!pip install gdal
import glob, os
from osgeo import gdal
from collections import defaultdict


# === 匯入衛星影像 ===
geotiff_path = '衛星影像路徑'
geotiff = [
    os.path.join(geotiff_path, f)
    for f in os.listdir(geotiff_path)
    if f.lower().endswith(('.tif', '.tiff'))
    ]


# === 按日期分組 ===
date_grouped_files = defaultdict(list)
for file in geotiff:
    base_name = os.path.basename(file)
    date_part = "_".join(base_name.split("_")[3:4])  # 提取日期部分
    date_grouped_files[date_part].append(file)


# === 逐日期合併影像 ===
for date, files in date_grouped_files.items():
    print(f"正在合併日期 {date} 的影像：{files}，共 {len(files)} 張")
    output_file = os.path.join(geotiff_path, f"S1_DV_SouthwestTaiwan_{date}_merge.tif")
    vrt_options = gdal.BuildVRTOptions(srcNodata=0) # 使用 gdal.BuildVRT 來合併影像，設定 NoData 為 0
    vrt = gdal.BuildVRT("/vsimem/temp.vrt", files, options=vrt_options)  # 在記憶體中建立 VRT
    gdal.Translate(output_file, vrt, format="GTiff") # 轉換 VRT 為 GeoTIFF
    vrt = None # 釋放 VRT 記憶體
    print(f"合併完成，輸出文件保存為：{output_file}")

In [ ]:
# @title Part 3 - 水域覆蓋率監測

# === 連接 Google Drive ===
"""
我們使用 Google Drive 作為衛星影像及地理空間資訊存取路徑，
故須先連結使用者Google Drive帳號以執行後續作業。
"""
from google.colab import drive
drive.mount('/content/drive')


# === 載入所需套件 ===
!pip install -q rasterio
import os, rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.mask import mask
from rasterio import features
from shapely.geometry import shape, box
from scipy.ndimage import binary_dilation

# === 匯入衛星影像及監測polygon ===
geotiff_path = '衛星影像路徑'
"""
geotiff = [
    os.path.join(geotiff_path, f)
    for f in os.listdir(geotiff_path)
    if f.lower().endswith(('.tif', '.tiff'))
    ]
"""
# 若想搜尋特定檔名可改用這段程式碼
target = "merge"
geotiff = [
    os.path.join(geotiff_path, f)
    for f in os.listdir(geotiff_path)
    if f.lower().endswith(('.tif', '.tiff')) and target in f
]


"""
上傳 GPKG 或 SHP (in EPSG:4326) 監測範圍檔案，
檔案須有一識別各polygon的名稱欄(如"ID"、"name")，作為監測範圍對應欄位。
"""
polygons_path = '監測範圍名稱及路徑'
gdf = gpd.read_file(polygons_path)
if gdf.crs != "EPSG:4326":
  gdf = gdf.to_crs("EPSG:4326")

# 篩選有交集影像
gdf_box = box(*gdf.total_bounds)
geotiff_filtered = []
for tif_path in geotiff:
    with rasterio.open(tif_path) as src:
        if box(*src.bounds).intersects(gdf_box):
            geotiff_filtered.append(tif_path)
print(f"🗺️ 篩選後共 {len(geotiff_filtered)} 筆影像包含監測範圍")


# === 計算水域覆蓋率 ===
# 指定極化及閾值
threshold = 0.029

# 暫存每個 polygon 的結果
records = []

# 衛星影像判釋迴圈
for geotifffile in geotiff_filtered:
    try:
        with rasterio.open(geotifffile) as src:
            filename = os.path.basename(geotifffile)

            # 嘗試從檔名提取日期資訊
            try:
                date_part = next((
                    p for p in filename.split('_')
                    if '-' in p and len(p.split('-')) == 3), None)
                if date_part:
                    year, month, day = date_part.split('-')
                else:
                    raise ValueError("未偵測到日期格式")
            except Exception:
                print(f"Skipping file {filename} — 無法解析日期格式")
                continue

            # 逐一處理每個 polygon
            for idx, row in gdf.iterrows():
                b = {"YEAR": year, "MONTH": month, "DAY": day}
                b['ID'] = row.get('ID', None) # 填入 polygon 名稱欄
                b['class'] = row.get('class', None)
                """
                若需要保留其他欄位，可在此新增，例如：
                b['WATERTYPE'] = row.get('WATERTYPE', None)
                b['GFISH'] = row.get('GFISH', None)
                b['area'] = row.get('area', None)
                """
                try:
                    # 使用 polygon 幾何範圍遮罩
                    out_image, out_transform = mask(
                        src,
                        [row.geometry],
                        crop=True,
                        filled=False,
                        all_touched=True
                    )

                    # 提取 VV 極化數值 (band 0)
                    band = out_image[0]

                    # 建立 polygon 邊界 mask，形成內部與邊界區域
                    polygon_mask = rasterio.features.geometry_mask(
                        [row.geometry],
                        transform=out_transform,
                        invert=True,
                        out_shape=band.shape
                        )
                    boundary_mask = rasterio.features.geometry_mask(
                        [row.geometry.boundary],
                        transform=out_transform,
                        invert=True,
                        out_shape=band.shape,
                        all_touched=True
                    )
                    internal_mask = polygon_mask & (~boundary_mask)

                    # 水域覆蓋率統計
                    boundary_data = band[boundary_mask]
                    boundary_pixels = boundary_data.size
                    internal_data = band[internal_mask]
                    internal_pixels = internal_data.size
                    internal_water_pixels = np.sum(internal_data <= threshold)
                    water_ratio = internal_water_pixels / internal_pixels if internal_pixels > 0 else np.nan

                    b.update({
                        'boundary_pixel': boundary_pixels,
                        'internal_pixel': internal_pixels,
                        'internal_water_pixel': internal_water_pixels,
                        'water_ratio': water_ratio
                    })
                    records.append(b)

                except Exception as e:
                    # print(f"⚠️ Polygon {b.get('ID', '未知')} in {filename} 發生錯誤：{type(e).__name__} — {e}")
                    continue

    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 無法開啟檔案 {geotifffile}：{e}")
        continue


# === 匯出結果 ===
df_vv = pd.DataFrame(records)
output_csv_path = '匯出名稱及路徑'
os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
df_vv.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print("\n✅ 水域覆蓋率分析完成！")
print(f"📁 輸出路徑：{output_csv_path}")
print(f"🗂️ 共匯出 {len(df_vv)} 筆紀錄，包含 {df_vv['ID'].nunique()} 個監測區域。")